In [1]:
import time
from snowflake.snowpark.session import Session
import configparser
 
import warnings
warnings.filterwarnings("ignore")
 
config = configparser.ConfigParser()
config.read("snowflake_connection.ini")
 
connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}
 
def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session
 
session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
df = session.table("CPG_DATA_ORIGINAL").to_pandas()
df_test = session.table("CPG_DATA_ORIGINAL").to_pandas()

In [3]:
#Changed location DONT RUN THE BELOW CELL

In [ ]:
import numpy as np

# Calculate the number of records for 'rural' and 'urban'
total_records = len(df)
rural_count = int(total_records * 0.7)
urban_count = total_records - rural_count

# Create the 'location' variable
location = ['rural'] * rural_count + ['urban'] * urban_count
np.random.shuffle(location)  # Shuffle to mix 'rural' and 'urban' values

# Add the 'location' variable to the DataFrame
df['location'] = location

# Save the updated DataFrame back to a CSV file
df.to_csv('CPG_DATA_LOCATION_CHANGE.csv', index=False)  # Replace 'updated_file.csv' with the desired output file name

# Print a success message
print("The 'location' variable has been added to the CSV file with 70% rural and 30% urban categories.")


In [ ]:
#df_location= df
#df_test_location= df

In [ ]:
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np

#import matplotlib.pyplot as plt
#%matplotlib inline



from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
# train dataset
#df = pd.read_csv("3.FMCG_data_location_changed.csv")

# test dataset
#df_test = pd.read_csv("3.FMCG_data_location_changed.csv")

In [ ]:
df_location.nunique().sort_values(ascending=False)

In [ ]:
df_location.drop(columns=['Ware_house_ID','WH_Manager_ID','Date'], inplace = True)
df_location['workers_num'].median()
df_location['approved_wh_govt_certificate'].mode()
df_location['approved_wh_govt_certificate'].fillna(df_location['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_location.isnull().sum()
df_location.nunique().sort_values(ascending= True)

In [ ]:
for i in df.columns[df_location.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_location[i].unique()))
    print(df_location[i].value_counts())
    print(' ')

In [ ]:
def count_outliers(df_location):
    outliers={}
    for i in df_location.columns:
        if pd.api.types.is_numeric_dtype(df_location[i]):
            q1 = df_location[i].quantile(0.25)
            q3 = df_location[i].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5*iqr
            upper = q3 + 1.5*iqr
            outliers[i] = df[(df[i] < lower) | (df_location[i] > upper)].shape[0]
        else:
            pass
    return outliers

In [ ]:
outlier_counts = count_outliers(df_location)
outlier_counts

In [ ]:
df_location['product_wg_ton'].corr(df_location['flood_proof'])
df_location['product_wg_ton'].corr(df_location['flood_impacted'])
df_location.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)

In [ ]:
def remove_outliers(df_location):
    num_df = df_location.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df.loc[df_no_outliers.index]

In [ ]:
df_location = remove_outliers(df_location)
df_location.shape

In [ ]:
df_location

In [ ]:
df_test_location.nunique().sort_values(ascending=False)

In [ ]:
df_test_location

In [ ]:
#df_test_location.drop(columns=['Ware_house_ID', 'WH_Manager_ID'], inplace = True)
#print(len(df_test_location[df_test_location.duplicated]))

In [ ]:
# print percentage of null values in each column

null_values_features = [i for i in df_test_location.columns if df_test_location[i].isnull().sum()>1]
for i in null_values_features:
    print(i, np.round(df_test_location[i].isnull().mean()*100, 2), "% missing values")

In [ ]:
df_test_location.drop(columns=['wh_est_year'], inplace = True)
df_test_location['workers_num'].median()
df_test_location['workers_num'].fillna(df_test_location['workers_num'].median(), inplace=True)
df_test_location['approved_wh_govt_certificate'].mode()
df_test_location['approved_wh_govt_certificate'].fillna(df_test_location['approved_wh_govt_certificate'].mode()[0], inplace=True)
df_test_location.isnull().sum()
df_test_location.nunique().sort_values(ascending= True)

In [ ]:
for i in df_test_location.columns[df_test_location.nunique()<=4]:
    print("Number of unique values in column ", i, ' :', len(df_test_location[i].unique()))
    print(df_test_location[i].value_counts())
    print(' ')

In [ ]:
df_test_location['product_wg_ton'].corr(df_test_location['storage_issue_reported'])

In [ ]:
#df_test_location.drop(columns=['flood_proof', 'flood_impacted'], inplace = True)

In [ ]:
def remove_outliers(df_test_location):
    num_df = df_test_location.select_dtypes(include=['int', 'float'])
    q1 = num_df.quantile(0.25)
    q3 = num_df.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_outliers = num_df[((num_df >= lower) & (num_df <= upper)).all(axis=1)]
    return df_test_location.loc[df_no_outliers.index]

In [ ]:
df_test_location = remove_outliers(df_test_location)

In [ ]:
# Numerical columns & categorical columns in train dataset

num_columns_train=[i for i in df_location.columns if df_location[i].dtypes != 'O']
cat_columns_train=[i for i in df_location.columns if df_location[i].dtypes == 'O']

# Numerical columns & categorical columns in test dataset

num_columns_test=[i for i in df_test_location.columns if df_test_location[i].dtypes != 'O']
cat_columns_test=[i for i in df_test_location.columns if df_test_location[i].dtypes == 'O']

In [ ]:
for i in cat_columns_train:
    df_location[i] = LabelEncoder().fit_transform(df_location[i])

for i in cat_columns_test:
    df_test_location[i] = LabelEncoder().fit_transform(df_test_location[i])

In [ ]:
# display number of unique values in each column
df_location.nunique().sort_values(ascending=False)

In [ ]:
columns_with_more_than_two_unique_values_train = df_location.columns[df_location.nunique()>2]

In [ ]:
columns_with_more_than_two_unique_values_test = df_test_location.columns[df_test_location.nunique()>2]

In [ ]:
df_location = df_location.drop(columns=['Location_type'])

In [ ]:
df_location.head()

In [ ]:
df_location_test = df_location 

In [ ]:

X_train = df_location.drop(['product_wg_ton', 'WH_regional_zone'], axis=1)
Y_train = df_location['product_wg_ton']


In [ ]:
X_test = df_location_test.drop(['product_wg_ton','WH_regional_zone'], axis=1)
X_test.head()

In [ ]:
#Linear Regression
LRmodel = LinearRegression()

In [ ]:
LRmodel.fit(X_train, Y_train)

In [ ]:
LR_y_predicted = LRmodel.predict(X_test)
LR_y_predicted


In [ ]:
predictions_location_nochange = pd.DataFrame(LR_y_predicted, columns =['product_wg_ton_pred_location_notchange']) 
predictions_location_nochange
predictions_location_nochange = predictions_location.reset_index()


In [ ]:
predictions_location_nochange

In [ ]:
df_test_location= df_test.reset_index()
df_test_location

In [ ]:
final_reco = pd.merge(df_test_location,predictions_location_nochange,on='index',how='inner')

In [ ]:
final_reco

In [ ]:
df_nochange_location=session.createDataFrame(
        final_reco.values.tolist(),
        schema=final_reco.columns.tolist())

In [ ]:
df_nochange_location.write.mode("overwrite").save_as_table("CPGWH_DEMO.FMCG.CPG_DATA_PRED_NOCHANGE_LOCATION1")

In [ ]:
df_nochange.write.mode("overwrite").save_as_table("CPGWH_DEMO.FMCG.CPG_DATA_PRED_NOCHANGE")

In [ ]:
predictions_location.to_csv('FMCG_data_original_predictions.csv', index=False)

In [ ]:
Y_test = df_test['product_wg_ton']
Y_test.head()

In [ ]:
# r2 score
lr_r2 = r2_score(Y_test, LR_y_predicted).round(3)
print('R2 score:  ', lr_r2)

# root mean squared error (RMSE)
lr_rmse = np.sqrt(mean_squared_error(Y_test, LR_y_predicted)).round(3)
print('Root Mean Squared Error:  ', lr_rmse) 

# mean absolute error (MAE)
lr_mae = mean_absolute_error(Y_test, LR_y_predicted).round(3)
print('Mean Absolute Error:  ', lr_mae)

In [ ]:
DTmodel = DecisionTreeRegressor(random_state = 0)

In [ ]:

DTmodel.fit(X_train, Y_train) 

In [ ]:
DT_predicted = DTmodel.predict(X_test)

In [ ]:
DT_predicted

In [ ]:
# r2 score
dt_r2 = r2_score(Y_test, DT_predicted).round(3)
print('R2 score:  ', dt_r2)

# root mean squared error (RMSE)
dt_rmse = np.sqrt(mean_squared_error(Y_test, DT_predicted)).round(3)
print('Root Mean Squared Error:  ', dt_rmse) 

# mean absolute error (MAE)
dt_mae = mean_absolute_error(Y_test, DT_predicted).round(3)
print('Mean Absolute Error:  ', dt_mae)

In [ ]:
RFmodel = RandomForestRegressor(n_estimators = 100, max_depth = 4, random_state = 0) 
RFmodel.fit(X_train, Y_train)
RF_predicted = RFmodel.predict(X_test)
RF_predicted

In [ ]:
# r2 score
rf_r2 = r2_score(Y_test, RF_predicted).round(3)
print('R2 score:  ', rf_r2)

# root mean squared error (RMSE)
rf_rmse = np.sqrt(mean_squared_error(Y_test, RF_predicted)).round(3)
print('Root Mean Squared Error:  ', rf_rmse) 

# mean absolute error (MAE)
rf_mae = mean_absolute_error(Y_test, RF_predicted).round(3)
print('Mean Absolute Error:  ', rf_mae)

In [ ]:

GBmodel = GradientBoostingRegressor()
GBmodel.fit(X_train,Y_train)
GB_predicted = GBmodel.predict(X_test)
GB_predicted

In [ ]:
gb_r2 = r2_score(Y_test, GB_predicted).round(3)
print('R2 score:  ', gb_r2)

# root mean squared error (RMSE)
gb_rmse = np.sqrt(mean_squared_error(Y_test, GB_predicted)).round(3)
print('Root Mean Squared Error:  ', gb_rmse) 

# mean absolute error (MAE)
gb_mae = mean_absolute_error(Y_test, GB_predicted).round(3)
print('Mean Absolute Error:  ', gb_mae)

In [ ]:
ax1 = plt.subplot2grid((2, 3), (0, 0))
plt.scatter(Y_test, LR_y_predicted, color='#ffdab9', alpha=0.3, label='Actual')
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--', color='#2f4f4f', label='Predicted')
plt.title('Linear Regression')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()


ax1 = plt.subplot2grid((2, 3), (0, 1))
plt.scatter(Y_test, DT_predicted, color='#ffdab9', alpha=0.3)
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--', color='#2f4f4f')
plt.title('Decision Tree')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

In [ ]:
ax1 = plt.subplot2grid((2, 3), (1, 1))
plt.scatter(Y_test, GB_predicted, color='#ffdab9', alpha=0.3)  # alpha controls point transparency for better visualization
plt.plot([min(Y_test), max(Y_test)], [min(Y_test), max(Y_test)], linestyle='--',color='#2f4f4f')
plt.title('Gradient Boosting')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

plt.show()